In [1]:
import os
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox

In [2]:
path = 'https://www.ptt.cc/bbs/Beauty/index.html'

try:
    driver = webdriver.Chrome(os.path.join(os.getcwd(),'chromedriver'))
except:
    errormessage = tk.Tk()
    errormessage.title("Error")
    errormessage.geometry('200x100')
    errormessage.resizable(0,0)  #鎖定視窗大小
    Label(errormessage,text = '').pack()
    Label(errormessage,text = '當前目錄下找不到 chromedriver').pack()
    Label(errormessage,text = '').pack()
    Button(errormessage,text='確認', command= lambda:errormessage.destroy()).pack()

    os._exit(0)

    # 客製化 webdriver 的檔案位置  (報錯:當前目錄下沒有chromedriver)  待測試


driver.get(path)
driver.find_element_by_class_name('btn-big').click()
# 18禁按鈕


In [3]:
save_path = os.getcwd() # 預設存檔在工作目錄
no_suck_link_list=[] #搜尋完顯示

In [4]:
def get_links(input_data):
    links_list = []
    links = input_data.find_elements_by_class_name('title')
    for i in range(len(links)):
        try:
            links_list.append((links[i].find_element_by_css_selector('a').get_attribute('href')))
        except NoSuchElementException:
            no_suck_link_list.append(links[i].text)
            
            
    if no_suck_link_list != []:
        messagebox.showinfo('注意','以下文章連結已失效:'+'\n'+'\n'.join((no_suck_link_list[i]) for i in range(len(no_suck_link_list))))                            
                            
    return links_list

In [5]:
def filter():
    driver.find_element_by_class_name('hl')

In [6]:
pic_link=[]

def search():   
    global num_choose,pic_link
    driver.current_url
    driver.find_element_by_name('q').clear()
    key_in = driver.find_element_by_name('q') #找到"搜尋文章"的欄位
    x = keyword.get()
    key_in.send_keys(x)
    key_in.submit()    
    pic_link = get_links(driver) # 搜尋後的 links    
    if pic_link == []:
        messagebox.showinfo(title="Error",message="查無資料")
        driver.back()
        driver.refresh()
    else:
        num_pic_link.set(len(pic_link))
        num_limit_text.set('請選擇要抓幾筆資料，最多'+str(num_pic_link.get())+'筆:')
        num_limit_btn.config(state='normal')        
        num_choose=Scale(window,label="",from_=1,to=num_pic_link.get(),
              orient = HORIZONTAL,
              length=400,
              digits=1,
              showvalue=1,
              tickinterval=1)
        num_choose.place(x=0,y=300,width=500,height=100)        
        return pic_link


In [7]:
def get_pic():
    hrefs = driver.find_elements_by_css_selector('a')

    images = []  # set to null at the start of every for loop
    for i in range(len(hrefs)):
        if hrefs[i].text[-4:] == '.jpg':
            images.append(hrefs[i].text.replace("://i.", "://"))

    
    name = title
    
    mix_path = os.path.join(save_path, name)
    #mix_path = os.path.join(os.getcwd(), name)
    if not os.path.isdir(mix_path):
        try:
            os.mkdir(mix_path)
        except:
            if not os.path.isdir(os.path.join(save_path, keyword.get())):
                os.mkdir(os.path.join(save_path, keyword.get()))

    pp = webdriver.Chrome(os.path.join(os.getcwd(),'chromedriver'))
    for j in range(len(images)):
        images_link = images[j]
        pp.get(images_link)
        photo = pp.find_element_by_css_selector('img')
        try:
            photo.screenshot(os.path.join(mix_path, str(j) + '.png'))
        except:
            pass
        
        # 存其他格式的圖片
        
        
        
    pp.quit()

In [8]:
def download_pic():
    global title
    x = num_choose.get()    
    for i in range(x):
        url = pic_link[i]
        webdriver.Chrome.get(driver, url)
        base_data = driver.find_elements_by_class_name('article-meta-value')
        if len(base_data) < 1:
            continue
        
        title = base_data[2].text
        
        if "/" in title:
            title = title.replace('/', '-')
        if "Re:" in title:
            title = title.replace('Re:', '')
        get_pic()
        
    webdriver.Chrome.get(driver, path)

In [9]:
def choose_path():
    global save_path
    save_path = filedialog.askdirectory()

In [10]:
def page_down():
    global pic_link
    driver.find_elements_by_class_name('btn')[3].click()
    #driver.current_url
    pic_link = get_links(driver)
    num_pic_link.set(len(pic_link))
    num_limit_text.set('請選擇要抓幾筆資料，最多'+str(num_pic_link.get())+'筆:')
    #set scale
    
def page_up():
    global pic_link
    driver.find_elements_by_class_name('btn')[4].click()
    #driver.current_url
    pic_link = get_links(driver)
    num_pic_link.set(len(pic_link))
    num_limit_text.set('請選擇要抓幾筆資料，最多'+str(num_pic_link.get())+'筆:')

In [11]:
window = tk.Tk()
window.title("表特版抓圖器")
window.geometry('500x500+100+100')
window.resizable(0,0)  #鎖定視窗大小

text_box = tk.Label(window,text = "請輸入搜尋條件:",
                    font=('Helvetica','20'))
text_box.place(x=30,y=10,width=300,height=50)

keyword = Entry(window,
                font=('Helvetica','30'))
keyword.place(x=30,y=70,width=300,height=90)

search_btn = Button(window,text="開始搜尋",
                    font=('Helvetica','20'),command=search)
search_btn.place(x=350,y=90,width=120,height=50)
    

exit_btn = Button(window,text = '離開',font=('Helvetica','20'),command = lambda: window.destroy())
exit_btn.place(x=390,y=10,width=80,height=50)
    
    
    
pageup_btn = Button(window,
                    text = '上一頁',
                    font=('Helvetica','20'),
                    command= page_up)
pageup_btn.place(x=30,y=170,width=150,height=50)

pagedown_btn = Button(window,
                    text = '下一頁',
                    font=('Helvetica','20'),
                    command= page_down)
pagedown_btn.place(x=180,y=170,width=150,height=50)


    
    
# <a class="btn wide" href="/bbs/Beauty/index3050.html">‹ 上頁</a>
    
# 推數篩選 選項    
    
    

num_pic_link = IntVar()
num_pic_link.set(0)
num_limit_text = StringVar()
num_limit_text.set('請選擇要抓幾筆資料，最多'+str(num_pic_link.get())+'筆:')

Label(window,
      textvariable=num_limit_text
      ,font=('Helvetica','20')).place(x=0,y=220,width=500,height=100)
# text vs textvariable


Button(window,text='請選擇存檔位置',font=('Helvetica','20'),
       command=choose_path).place(x=50,y=400,width=200,height=50)



num_limit_btn = Button(window,text='開始抓圖',font=('Helvetica','20'),
                       command=download_pic)
num_limit_btn.place(x=300,y=400,width=120,height=50)
num_limit_btn.config(state='disabled')



window.mainloop()

1 推數篩選

2 各種圖片類型 ---> 遇到困難

3 抓第幾筆(選擇性)

V 切換頁面

V 離開程式

5 轉成EXE執行檔
  
4 排版

** 抓圖時程式沒有回應 ----> 鎖定視窗，彈出"抓圖中"字樣

** 有時候標題相同會抓不到

** 跳過某幾筆資料